In [1]:
import ddm_data_simulation as ddm_sim
import scipy as scp
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.neighbors import KernelDensity
#import kde_class
import multiprocessing as mp
import psutil
import pickle 
import os
import re

import kde_training_utilities as kde_util
import kde_class as kde

In [ ]:
# Load some file and extract column names to be kept going forward
test = pickle.load(open(os.getcwd() + '/data_storage/kde/flexbound/kde_base_dat/kde_ddm_flexbound_fff84ea0681f11e98948897ab099f7d4.pickle', 'rb'))
col_names = list(test)
test_frame = pd.DataFrame(columns = ['cols_to_keep', 'kde_type', 'bandwidth_rule', 'mixture'])
test_frame.loc[0] = [col_names[:8], test['kde_type'][0], test['bandwidth_rule'][0], test['mixture'][0]]

In [ ]:
# Write meta file that stores the columns we are interested in in a csv
test_frame.to_csv(os.getcwd() + '/data_storage/kde/kde_base_dat/meta_data.csv', index = False)

In [3]:
# Initializations
#meta = pd.read_csv(os.getcwd() + '/data_storage/kde/kde_base_dat/meta_data.csv')
simulator = 'kde_ddm_flexbound'
#to_folder = os.getcwd() + '/data_storage/kde/kde_base_dat/' # + simulator + '_' + 'test_full' 
test_folder = os.getcwd() + '/data_storage/kde/flexbound/' + simulator + '_' + 'reduced_files' 
final_folder = os.getcwd() + '/data_storage/kde/flexbound/' + simulator + '_' + 'final_train_test'
#file_list = os.listdir(to_folder)

In [ ]:
# Make reduced data (dropping unnecessary column w.r.t network training)
cnt = 0
for file_ in file_list:
    if file_ != 'meta_data.csv':
        my_mat = pickle.load(open(to_folder + '/' + file_, 'rb'))
        my_mat = my_mat[eval(meta['cols_to_keep'][0])]
        pickle.dump(my_mat, open(test_folder + '/' + file_, "wb"))
        cnt += 1
    print(cnt)

In [4]:
# Make train test split now
file_list = os.listdir(test_folder)
file_list_with_dir = [test_folder + '/' + file for file in file_list]
df = pd.concat([pd.read_pickle(fp) for fp in file_list_with_dir], ignore_index = True)

In [5]:
# Get training and test labels
train_id = np.random.choice(a = [True, False], 
                            size = df.shape[0], 
                            replace = True, 
                            p = [0.8, 0.2])
test_id  = np.invert(train_id)

In [10]:
# Store training and test data to file 
df.iloc[train_id, :7].to_pickle(final_folder + '/train_features.pickle' , protocol = 4)
df.iloc[test_id, :7].to_pickle(final_folder + '/test_features.pickle', protocol = 4)
df.iloc[train_id, 7].to_pickle(final_folder + '/train_labels.pickle', protocol = 4)
df.iloc[test_id, 7].to_pickle(final_folder + '/test_labels.pickle', protocol = 4)

In [9]:
df.iloc[0,:7]

v        -2.038474
a         1.503047
w         0.552430
c1        2.515306
c2        0.665409
rt        0.804790
choice   -1.000000
Name: 0, dtype: float64